In [ ]:
# =============================================================================
# Created By:     Kai
# Created School: Franz-Oberthuer-Schule Wuerzburg
# Created Email:  metzgerkai@franz-oberthuer-schule.de
# Created Date:   Fri December 06 18:08 UTC 2024
# Version:        1.0
# =============================================================================
"""The Module has been build for training the symbols dataset with images + 
ground truth on a Raspberry Pi 5 with a standard USB camera. An image with a 
resolution of 640px x 480px can be recorded with the Python script 
create-symbols-dataset.py.
The files in the dataset <2023 symbols> should be ordered in the following
manner:             > explanation
- 2023 symbols      
  - dataset3        > dataset version (dataset1 - 3 where used in the project)
    - data          > images (.png files)
    - gt            > ground truth (.txt files with class labels 1-4)
"""

# =============================================================================
# Import
# =============================================================================
import cv2
import os
import glob
import numpy as np
from tensorflow.keras import layers, models, utils, callbacks
import matplotlib.pyplot as plt

# To get reproducable results with the same training setting random seed
SEED = 42
np.random.seed(SEED)

# Flags
plots_on = True
test_on_specific_image = True

In [ ]:
# =============================================================================
# Define functions
# =============================================================================
# Change image size and convert to grayscale images
def pic_prep (image, x, y):
  image = cv2.resize(image, (y,x)) # change image size
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert to grayscale
  image = image / 255 # image normalization
  return image

# Shuffle images
def unison_shuffle(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

# Save training info in file.txt
def write_infotext(filepath,
                   image_size,
                   num_neurons_per_layer,
                   batch_size,
                   epochs,
                   history):
    with open(filepath + 'info.txt','w') as txt:
        txt.write('------------------- SPECS -------------------\n'
                  '\n'
                  'IMG Resolution: ' f'{image_size}\n'
                  'Neurons per Layer: ' f'{num_neurons_per_layer}\n'
                  'Batch Size: ' f'{batch_size}\n'
                  'Epochs: ' f'{epochs}\n'
                  '\n'
                  '##############################################\n'
                  '\n'
                  '\n'
                  '------------------- HISTORY -------------------\n'
                  '\n'
                  f'{history}\n')

# todo: only show output of training for every 10 epochs
# Add callback to only print loss etc. after e.g. 10 epochs:
# https://stackoverflow.com/questions/44931689/how-to-disable-printing-reports-after-each-epoch-in-keras
class callback_print(callbacks.Callback):
    SHOW_NUMBER = 10
    counter = 0
    epoch = 0

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch = epoch

    def on_train_batch_end(self, batch, logs=None):
        if self.counter == self.SHOW_NUMBER or self.epoch == 1:
            print('Epoch: ' + str(self.epoch) + ' loss: ' + str(logs['loss']))
            if self.epoch > 1:
                self.counter = 0
        self.counter += 1

In [ ]:
# =============================================================================
# Function with some hypterparameters to change for comparison
# =============================================================================
def train_signs_model(num_training=0,
                      image_size=[32, 28],
                      num_neurons_per_layer=[64,32,16,13],
                      batch_size=4,
                      epochs=300,
                      counter=1):
    # =============================================================================
    # Declare variables
    # =============================================================================
    img_size_x = image_size[0]
    img_size_y = image_size[1]
    img_dim = img_size_x * img_size_y
    data_index = 0

    # Change the following path to your dataset path
    path = '/home/pi/Documents'
    #path = '/home/metzgeka/ml_project/datasets'
    dataset_dir = path + '/Verkehrszeichenerkennung/projekt_hk_kt/dataset/'
    labels = [label for label in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, label))]
    checkpoint_filepath = path + '/Verkehrszeichenerkennung/projekt_hk_kt/chpt/' + f'{counter}'
    os.makedirs(checkpoint_filepath, exist_ok=True)
    img_path = glob.glob(dataset_dir + '*/*.jpg')
    anz_data = len(img_path)

    # =============================================================================
    # Get number of data (image/label) 
    # =============================================================================
    anz_data = int(anz_data) - 1
    dataset = np.zeros((anz_data, img_size_x, img_size_y), dtype=float)
    ground_truth = np.zeros((anz_data), dtype=str)
    #print("dataset size:", anz_data)

    # =============================================================================
    # Callback for training
    # =============================================================================
    model_filepath=checkpoint_filepath + str(num_training) + '-chpt.model.keras'

    model_checkpoint_callback = callbacks.ModelCheckpoint(
        model_filepath,
        save_weights_only=False,
        monitor='val_accuracy',
        mode='auto',
        save_best_only=True,
        verbose = 1)
    
    # =============================================================================
    # Read dataset
    # =============================================================================
    for i in range(0, anz_data, 1):
        image = cv2.imread(img_path[i])                         # read image
        image = pic_prep(image, img_size_x, img_size_y)         # prepare image
        dataset[i,:,:] = image                                  # 2d-image to 3d-array
        gt_path = os.path.normpath(img_path[i]).split(os.sep)   #
        ground_truth[i] = gt_path[-2]                           # ground truth for all images

    # =============================================================================
    # Prepare dataset: train set (80%) and test set (20%)
    # =============================================================================
    dataset = dataset.reshape(anz_data, img_dim) # convert into 2d array (all pixel in one row)
    ground_truth = ground_truth.reshape(anz_data, 1)

    dataset, ground_truth = unison_shuffle(dataset, ground_truth)

    trainset = np.random.choice(dataset.shape[0],
                                int(dataset.shape[0]*0.80), 
                                replace=False)
    train_data = dataset[trainset,:]
    train_gt = ground_truth[trainset]
    train_gt = utils.to_categorical(train_gt, 13)

    testset = np.delete(np.arange(0, len(ground_truth) ), 
                        trainset) 
    test_data = dataset[testset,:]
    test_gt = ground_truth[testset]
    test_gt = utils.to_categorical(test_gt, 13)
    #print(ground_truth)
    #print(dataset)
    #print(trainset)
    #print(train_data)
    #print(train_gt)

    # =============================================================================
    # Create neural network with e.g. 4 layers and (64, 32, 16, 4) neurons per layer.
    # =============================================================================
    model = models.Sequential()
    model.add(layers.Dense(num_neurons_per_layer[0], input_dim=img_dim,activation='relu'))
    model.add(layers.Dense(num_neurons_per_layer[1], activation='relu'))
    model.add(layers.Dense(num_neurons_per_layer[2], activation='relu'))
    model.add(layers.Dense(num_neurons_per_layer[3], activation='sigmoid'))
    model.compile(loss='mean_squared_error', 
                  optimizer='adam', 
                  metrics=['accuracy'])

    # =============================================================================
    # Train the neuronal network e.g. for e.g. 300 epochs.
    # =============================================================================
    history = model.fit(train_data, 
                        train_gt, 
                        batch_size=batch_size, 
                        epochs=epochs, 
                        verbose=0, 
                        shuffle=True, 
                        validation_data=(test_data, test_gt), 
                        callbacks=[model_checkpoint_callback])
    
    # =============================================================================
    # Plot train and val accuracy.
    # =============================================================================
    if plots_on:
        plt.plot(history.history['accuracy'], label='accuracy')
        plt.plot(history.history['val_accuracy'], label='val_accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend(loc='lower right')
        plt.savefig(checkpoint_filepath + 'acc.png', dpi=300, bbox_inches='tight')
        plt.show()

        # =========================================================================
        # Plot train and val loss.
        # =========================================================================
        plt.plot(history.history['loss'], label='loss')
        plt.plot(history.history['val_loss'], label='val_loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend(loc='lower left')
        plt.savefig(checkpoint_filepath + 'loss.png', dpi=300, bbox_inches='tight')
        plt.show()
    
    if test_on_specific_image:
        # =========================================================================
        # Load pretrained dataset weights to e.g. test on new (unseen) data.
        # =========================================================================
        model.load_weights(model_filepath)   

        # =========================================================================
        # Test dataset on xxx.
        # =========================================================================
        score = model.evaluate(test_data, test_gt, verbose=1)
        print('Test score:', score[0])
        print('Test accuracy:', score[1])       

        # =========================================================================
        # Testing on a single image.
        # =========================================================================
        
        data_pred = np.zeros((1, img_size_x, img_size_y), dtype=float)
        img_pred = cv2.imread(path + '/Verkehrszeichenerkennung/projekt_hk_kt/dataset/Achtzig/00021_00029..jpg')
        img_pred = pic_prep(img_pred, img_size_x, img_size_y)
        data_pred[0,:,:] = img_pred
        data_pred = data_pred.reshape(1,img_dim)
        result = model.predict(data_pred)
        result = np.round(result, decimals=2)
        print('Classes:', labels)
        print('Probability for class in percent', result)
        
        """
        # =============================================================================
        # Output class: 
        # translate class label (0,1,2,3) to class (cross, circle, triangle, square).
        # =============================================================================
        max_res = 0
        res_index = 13
        for i in range(0, 13, 1):
            if result[0,i] > max_res:
                max_res = result[0,i]
                res_index = i

        if res_index == 0:
            print('Cross detected!')
        elif res_index == 1:
            print('Circle detected!')
        elif res_index == 2:
            print('Triangle detected!')
        elif res_index == 3:
            print('Square detected!')
        elif res_index == 4:
            print('Error!')
        """
    write_infotext(checkpoint_filepath,
                   image_size,
                   num_neurons_per_layer,
                   batch_size,
                   epochs,
                   history)
    
    counter += 1

In [ ]:
# =============================================================================
# Train multiple networks with differend image scales and neurons per layer
# =============================================================================

image_dims = [[64, 56]]
"""
              [32, 28],
              [32, 28],
              [32, 28],
              [32, 28]]
"""
neurons_layers = [[256,32,16,13]]
"""
                  [128,32,16,13],
                  [64,32,16,13],
                  [32,32,16,13],
                  [16,32,16,13]]
"""

# Train multiple times with different hyper parameters (dim img size, neurons per layer)
i = 1
for img_size, num_neurons in zip(image_dims, neurons_layers):
    print("Training", i, "with image size", img_size, "and", num_neurons, "neurons per layer")
    train_signs_model(i, img_size, num_neurons, 4, 10, 1)
    print("#################################################\n")
    i+=1